<a href="https://colab.research.google.com/github/arnabsinha99/MListheroxx/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import helper
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = "retina"

#SINCE IMAGENET IS HEAVY TO DOWNLOAD  I HAVE USED CIFAR-10 data on Alexnet, which is not recommended but for convenience of internet during Lockdown!!

In [61]:
transform = transforms.Compose([#transforms.ToPILImage(),
                                transforms.Resize((227,227)),
                                transforms.ToTensor(),
                                ])
train = datasets.CIFAR10(root = 'data',train = True, transform = transform, download = True)
test = datasets.CIFAR10(root = 'data',train = False, transform = transform, download = True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
num_train = len(train)
indices = list(range(num_train))
np.random.shuffle(indices)

train_val_split = 0.2 # 20% validation set out of training set

split = int(np.floor(train_val_split*num_train))
train_idx,valid_idx = indices[split:],indices[:split]

trainSampler = SubsetRandomSampler(train_idx)
validSampler = SubsetRandomSampler(valid_idx)

num_workers = 0
b_size = 100

In [0]:
train_data = torch.utils.data.DataLoader(train, b_size, sampler = trainSampler, num_workers=num_workers)
test_data = torch.utils.data.DataLoader(test, b_size, num_workers=num_workers)
valid_data = torch.utils.data.DataLoader(train, b_size, sampler = validSampler, num_workers=num_workers)

In [0]:
# dataiter = iter(train_data)

# images, labels = next(dataiter)
# images = images.numpy()

In [0]:
# num_train

In [0]:
# fig = plt.figure(figsize = (25,4))
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
            'dog', 'frog', 'horse', 'ship', 'truck']
# def imshow(img):
#     #img = img / 2 + 0.5  # unnormalize
#     plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

# for idx in range(b_size):
#   ax = fig.add_subplot(2, 10, idx+1, xticks = [],yticks = [])
#   imshow(images[idx])
#   ax.set_title(classes[labels[idx].item()])

In [0]:
# def conv_batch(images,b_size):
#   im = []
#   for idx in range(b_size):
#     im.append(((images[idx][0]+images[idx][1]+images[idx][2])/3).flatten())
#   return np.array(im).reshape((20,227,227))

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class BasicNet(nn.Module):
  def __init__(self, num_classes = 10):
    super(BasicNet, self).__init__()
    
    self.net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
    )

    self.classifier = nn.Sequential(
        nn.Dropout(p=0.5,inplace = True),
        nn.Linear(in_features = (256*6*6),out_features = 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5,inplace = True),
        nn.Linear(in_features = 4096,out_features = 4096),
        nn.ReLU(),
        nn.Linear(in_features=4096, out_features=num_classes)
    )

    self.init_bias()
  
  def init_bias(self):
    for layer in self.net:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean=0, std=0.01)
        nn.init.constant_(layer.bias, 0)
    nn.init.constant_(self.net[4].bias, 1)
    nn.init.constant_(self.net[10].bias, 1)
    nn.init.constant_(self.net[12].bias, 1)
  
  def forward(self,x):
    x = self.net(x)

    x = x.view(-1,256*6*6)
    return self.classifier(x)

In [0]:
NUM_EPOCHS = 5  # original paper
BATCH_SIZE = 100
MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01
IMAGE_DIM = 227  # pixels
NUM_CLASSES = 10  # 1000 classes in original paper for ImageNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
a = BasicNet(num_classes=NUM_CLASSES).to(device)

In [67]:
train_on_gpu = torch.cuda.is_available()
train_on_gpu

True

In [0]:
#Optimizer
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(a.parameters(),lr = 0.01)
optimizer = optim.SGD(a.parameters(),lr = LR_INIT,momentum = MOMENTUM, weight_decay=LR_DECAY)

In [0]:
lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma = 0.1)

In [70]:
#Training

valid_loss_min = np.Inf

for epoch in range(NUM_EPOCHS):
  train_loss = 0.0
  valid_loss = 0.0
  
  a.train()
  #b = 0
  for imgs,classes in train_data:

    #print("Training batch {} ...".format(b))
    imgs,classes = imgs.to(device),classes.to(device)

    output = a(imgs)
    loss = criterion(output,classes)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    train_loss += loss.item()*imgs.size(0)
    #b+=1

  print("Training done for Epoch {}".format(epoch))

  print('Evaluation started...')

  a.eval()
  for imgs, target in valid_data:
    imgs,target = imgs.to(device),classes.to(device)
    
    output = a(imgs)
    loss = criterion(output, target)

    valid_loss += loss.item()*imgs.size(0)
    
  train_loss = train_loss/len(train_data.sampler)
  valid_loss = valid_loss/len(valid_data.sampler)
        
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
    valid_loss_min = valid_loss
  lr_scheduler.step()

Training done for Epoch 0
Evaluation started...
Epoch: 0 	Training Loss: 2.306245 	Validation Loss: 2.305328
Validation loss decreased (inf --> 2.305328).  Saving model ...
Training done for Epoch 1
Evaluation started...
Epoch: 1 	Training Loss: 2.302716 	Validation Loss: 2.303598
Validation loss decreased (2.305328 --> 2.303598).  Saving model ...
Training done for Epoch 2
Evaluation started...
Epoch: 2 	Training Loss: 2.302569 	Validation Loss: 2.303017
Validation loss decreased (2.303598 --> 2.303017).  Saving model ...
Training done for Epoch 3
Evaluation started...
Epoch: 3 	Training Loss: 2.302585 	Validation Loss: 2.303158
Training done for Epoch 4
Evaluation started...
Epoch: 4 	Training Loss: 2.302583 	Validation Loss: 2.302472
Validation loss decreased (2.303017 --> 2.302472).  Saving model ...


In [78]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

a.eval()
for data, target in test_data:
    data,target = imgs.to(device),target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = a(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(b_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_data.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 2.302615

Test Accuracy of airplane:  0% ( 0/1000)
Test Accuracy of automobile:  0% ( 0/1000)
Test Accuracy of  bird: 100% (1000/1000)
Test Accuracy of   cat:  0% ( 0/1000)
Test Accuracy of  deer:  0% ( 0/1000)
Test Accuracy of   dog:  0% ( 0/1000)
Test Accuracy of  frog:  0% ( 0/1000)
Test Accuracy of horse:  0% ( 0/1000)
Test Accuracy of  ship:  0% ( 0/1000)
Test Accuracy of truck:  0% ( 0/1000)

Test Accuracy (Overall): 10% (1000/10000)
